In [1]:
import psycopg2
import pandas as pd
import json
import datetime

In [2]:
aujourd= datetime.datetime.today()
aujourd
print(f"{aujourd:%d %m  %Y}")

26 07  2022


- Afficher pour chaque annexe, la liste des champs (./app/config.py)
- Pour chaque annexe, créer la requête pour récupérer les données concernant l'annexe de manière variabilisé
- Créer une fonction qui prend en argument une connexion à la base de données, l'année, le type d'annexe et la nature de décision. Cette fonction fait une requête à la base de donnée avec ces arguments. En retour elle renvoie un dataframe pandas 
- Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.
- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

In [2]:
def get_index(nom):
    data = {"a": [2, 1, 10], "b": [5,3,5] }
    l = data[nom]
 
    return l

In [3]:
get_index(nom="a") 

[2, 1, 10]

# Affichage de la liste des champs


In [3]:
# 1- Les données de toutes les annexes
import app.backend.config as conf
data = conf.CHAMPS_ANNEXES
data

{'DATA_PRES_CROISEE': ['RefFonc',
  'ChapitreNature',
  'Section',
  'InfoOpe',
  'CodRD',
  'OpBudg',
  'MtBudgPrec',
  'MtRARPrec',
  'MtPropNouv',
  'MtPrev',
  'CredOuv',
  'MtReal',
  'MtRAR3112',
  'MtCumul',
  'MtCumulCA',
  'MtMdtTitreEmis',
  'MtSup'],
 'DATA_EMPRUNT': ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv'

In [ ]:
for key, value in data.items() :
    print(value)

In [ ]:
for item in data:
    print(item)

In [6]:
# 2 - Récupérer les données concernant l'annexe de manière variabilisé
def annexe(nom):
    donnees = data[nom]
    return donnees

In [ ]:
# 2.1 - Afficher les donnees de DATA_EMPRUNT
annexe(nom="DATA_EMPRUNT")

In [ ]:
# 2.2 - Afficher les donnees de toutes les annexes
for i in data:
    donnee = annexe(nom=i)
    print(donnee)

In [13]:
# 3- connexion à la base de donnée et  retour d'un dataframe pandas 
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")


def get_data(connexion):
    query= """SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = 'DATA_EMPRUNT'
    AND documentbudgetaire.exercice = '2020'
    AND documentbudgetaire.nature_dec = '09' 
        LIMIT 100 """
    dat = pd.read_sql_query(query, conn)
    # conn.close()
    return dat
get_data(connexion=conn)

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,206813,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",35999,21510065200019,"[DATA_EMPRUNT, DATA_SIGNATURE, DATA_SOMMAIRE]",35999,COMMUNE DE BOUCHY-SAINT-GENEST,51071,M14-M14_COM_INF500,...,None,1,P,None,"[{""Nature"": ""6061"", ""LibCpte"": ""Fournitures no...",21510065200019,21510065200019,COMMUNE DE BOUCHY-SAINT-GENEST,Commune,51
1,206816,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",36000,21700380500017,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",36000,BUDGET COMMUNE,70380,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",21700380500017,21700380500017,NEUREY EN VAUX,COMMUNE,70
2,206824,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""F"", ""...",36001,20008298000117,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",36001,LOTISSEMENT RESIDENCE 3 VAL-COUESNON,None,M14-M14_COM_500_3500,...,None,1,A,20008298000018,"[{""Nature"": ""7015"", ""ArtSpe"": ""false"", ""CodRD""...",20008298000117,20008298000117,COMMUNE DE VAL-COUESNON,Commune,35
3,206890,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""747431.45""}]",36012,20007415100016,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",36012,CIAS - Budget principal CIAS,None,M14-M14_CCAS_SUP3500,...,None,3,P,None,"[{""Nature"": ""023"", ""Fonction"": ""01"", ""ContNat""...",20007415100016,20007415100016,CIAS Coeur & Coteaux du Cominges,CCAS,31
4,206898,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",36013,21850001500100,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",36013,PORT (301) M 4 -99,085023,M4-M4,...,None,1,A,21850001500019,"[{""Nature"": ""002"", ""ArtSpe"": ""false"", ""CodRD"":...",21850001500100,21850001500100,PORT (301) M 4 -99,PORT (301) M 4 -99,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,209760,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",36504,21710072600017,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",36504,Principal,071047072,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ArtSpe"": ""false"", ""CodRD"":...",21710072600017,21710072600017,Budget principal de Cersot,Budget principal de Cersot,71
96,209767,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",36505,21630084800019,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_FISCALITE, ...",36505,COMMUNE DE CHANONAT,None,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""60612"", ""LibCpte"": ""\u00c9nergie ...",21630084800019,21630084800019,COMMUNE,Commune,None
97,209774,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""102690.90""}]",36506,20005714900060,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",36506,Budget Assainissement,39177,M4-M49_D,...,None,1,A,20005714900011,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",20005714900060,20005714900060,Commune de HAUTEROCHE,COMMUNE,39
98,209791,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",36509,20009142900023,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",36509,Budget Assainissement 12501,39,M4-M49_D,...,None,1,P,None,"[{""Nature"": ""6226"", ""ContNat"": ""011"", ""ArtSpe""...",20009142900023,20009142900023,SIE des 3 Rivières,SMFERME,39


In [15]:
df_emprunt = get_data(connexion=conn)

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [16]:
df_emprunt

,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,209760,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",36504,21710072600017,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",36504,Principal,071047072,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ArtSpe"": ""false"", ""CodRD"":...",21710072600017,21710072600017,Budget principal de Cersot,Budget principal de Cersot,71
1,209767,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",36505,21630084800019,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_FISCALITE, ...",36505,COMMUNE DE CHANONAT,None,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""60612"", ""LibCpte"": ""\u00c9nergie ...",21630084800019,21630084800019,COMMUNE,Commune,None
2,209774,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""102690.90""}]",36506,20005714900060,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",36506,Budget Assainissement,39177,M4-M49_D,...,None,1,A,20005714900011,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",20005714900060,20005714900060,Commune de HAUTEROCHE,COMMUNE,39
3,209791,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",36509,20009142900023,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",36509,Budget Assainissement 12501,39,M4-M49_D,...,None,1,P,None,"[{""Nature"": ""6226"", ""ContNat"": ""011"", ""ArtSpe""...",20009142900023,20009142900023,SIE des 3 Rivières,SMFERME,39
4,209802,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",36511,21320180900034,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_VENTIL...",36511,Transport scolaire LAGRAULET,32180,M4-M43_A,...,None,1,P,None,"[{""Nature"": ""1641"", ""ArtSpe"": ""false"", ""CodRD""...",21320180900034,21320180900034,Transport scolaire LAGRAULET,Transport scolaire LAGRAULET,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,212702,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",37033,21120156100068,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_ORGANI...",37033,Budget assainissement,12156,M4-M49_A,...,None,1,A,21120156100019,"[{""Nature"": ""002"", ""ContNat"": ""002"", ""ArtSpe"":...",21120156100068,21120156100068,COMMUNE DE MONTPEYROUX,COMMUNE,12
96,212708,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",37034,21720024500066,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",37034,SERVICE D'ASSAINISSEMENT- LA BAZOGE,72024,M4-M49_A,...,None,1,A,21720024500017,"[{""Nature"": ""2762"", ""ArtSpe"": ""false"", ""CodRD""...",21720024500066,21720024500066,COMMUNE DE LA BAZOGE,Commune,72
97,212715,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""1071.78""}]",37035,20006406100282,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",37035,PARRACHEE VANOISE,73290,M14-M14_COM_500_3500,...,None,1,A,20006406100019,"[{""Nature"": ""757"", ""ArtSpe"": ""false"", ""CodRD"":...",20006406100282,20006406100282,COMMUNE DE VAL CENIS,Commune,73
98,212732,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""909461.25""}]",37038,21390297600017,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",37038,BUDGET COMMUNAL M 14,39297,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6238"", ""ContNat"": ""011"", ""ArtSpe""...",21390297600017,21390297600017,COMMUNE DE LONGCHAUMOIS,COMMUNE,39


In [18]:
# 3.1 -
list(df_emprunt.columns)

['id',
 'type_annexe',
 'json_annexe',
 'fk_id_document_budgetaire',
 'siret_etablissement',
 'list_annexes',
 'id',
 'libelle',
 'code_insee',
 'nomenclature',
 'exercice',
 'nature_dec',
 'num_dec',
 'nature_vote',
 'type_budget',
 'id_etabl_princ',
 'json_budget',
 'fk_siret_collectivite',
 'siret_coll',
 'libelle_collectivite',
 'nature_collectivite',
 'departement']

In [19]:
# 3.2 -
df_emprunt.json_annexe

0     [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
1     [{"CodTypEmpr": "01", "CodProfilAmort": "C", "...
2       [{"CodTypEmpr": "10", "RReelFon": "102690.90"}]
3     [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
4     [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
                            ...                        
95    [{"CodTypEmpr": "01", "CodProfilAmort": "C", "...
96    [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
97        [{"CodTypEmpr": "10", "RReelFon": "1071.78"}]
98      [{"CodTypEmpr": "10", "RReelFon": "909461.25"}]
99    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
Name: json_annexe, Length: 100, dtype: object

In [ ]:
# 3.3 -
for item in df_emprunt.json_annexe:
    print(type(item))

In [21]:
# 3.4 - avec f string
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

def get_data(connexion, annexe, year, numero_docbudge):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '{numero_docbudge}' 
        LIMIT 5 """
    dat = pd.read_sql_query(query, conn)
    return dat
get_data(connexion=conn, annexe='DATA_EMPRUNT', year=2019, numero_docbudge= '09')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,212919,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",37072,20007053000064,"[DATA_EMPRUNT, DATA_PATRIMOINE, DATA_PERSONNEL...",37072,IMMEUBLES PRODUCTIFS DE REVENUS,014,M14-M14_COM_SUP3500,...,None,3,A,20007053000015,"[{""Nature"": ""7472"", ""Fonction"": ""71"", ""ArtSpe""...",20007053000064,20007053000064,C.C LE GRAND AUTUNOIS MORVAN,,71
1,212927,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""65269.52""}]",37074,21600483800035,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",37074,PEROY LES GOMBRIES EAU ET ASS.,489,M4-M49_D,...,None,1,P,None,"[{""Nature"": ""6226"", ""ContNat"": ""011"", ""ArtSpe""...",21600483800035,21600483800035,SERVICE DES EAUX PEROY,COMMUNE,48
2,213167,DATA_EMPRUNT,"[{""CodTypEmpr"": ""09"", ""CodProfilAmort"": ""P"", ""...",37112,24060058500014,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_TIERS, DA...",37112,BUDGET PRINCIPAL,None,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""001"", ""Fonction"": ""01"", ""ContNat""...",24060058500014,24060058500014,COMMUNAUTE D'AGGLO. SOPHIA ANTIPOLIS,COMMUNE dont la population est de 3500 habitan...,None
3,213339,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",37135,24570127100059,"[DATA_EMPRUNT, DATA_AMORTISSEMENT_METHODE, DAT...",37135,BUDGET ANNEXE GESTION LOCATIVE,591,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""1641"", ""Fonction"": ""90"", ""ArtSpe""...",24570127100059,24570127100059,BUDGET ANNEXE - GESTION LOCATIVE (M57),Commune,95
4,213983,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",37243,21240158200011,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_FORMATION, ...",37243,DUSSAC,24158,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6336"", ""ContNat"": ""012"", ""ArtSpe""...",21240158200011,21240158200011,DUSSAC,Commune,24


Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.

In [26]:
nat_de = ['01', '02', '03','09']
value_de = ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"]
replace_typebud = zip(["p","A"],
                    ["Principal", "Annexe"])
dict_replace = {'nat_dec': dict(zip(nat_de, value_de)),
                 "nat": dict(replace_typebud)}
print(dict_replace)

{'nat_dec': {'01': 'Budget primitif', '02': 'Décision modificative', '03': 'Budget supplémentaire', '09': 'Compte administratif'}, 'nat': {'p': 'Principal', 'A': 'Annexe'}}


In [28]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace          

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [ ]:
list(df_emprunt.columns)

In [29]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(df_emprunt.columns) if item not in interesting_columns]
df = df_emprunt.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
df_min
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",21710072600017,Principal,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21710072600017,Budget principal de Cersot,Budget principal de Cersot
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",21630084800019,COMMUNE DE CHANONAT,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21630084800019,COMMUNE,Commune
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""102690.90""}]",20005714900060,Budget Assainissement,M4-M49_D,2020,Compte administratif,1,Annexe,20005714900060,Commune de HAUTEROCHE,COMMUNE
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",20009142900023,Budget Assainissement 12501,M4-M49_D,2020,Compte administratif,1,Principal,20009142900023,SIE des 3 Rivières,SMFERME
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",21320180900034,Transport scolaire LAGRAULET,M4-M43_A,2020,Compte administratif,1,Principal,21320180900034,Transport scolaire LAGRAULET,Transport scolaire LAGRAULET


In [30]:
EMPRUNT_FIELD = ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv',
  'IndSousJacentDtVote',
  'IndexTxVariDtVote',
  'IndexTxVariInit',
  'IndexTxVariReneg',
  'IndiceCouv',
  'IndiceEmpr',
  'IndiceEmprDtVote',
  'LibCpte',
  'LibOrgCoContr',
  'LibOrgaPreteur',
  'MPrimeRecueCouv',
  'MtCRDCouvert',
  'MtCRDRefin',
  'MtCapitalExer',
  'MtCapitalReamenage',
  'MtCapitalRestDu_01_01',
  'MtCapitalRestDu_31_12',
  'MtCharges',
  'MtCommCouv',
  'MtCouv',
  'MtCouvert',
  'MtEmprOrig',
  'MtEmprReneg',
  'MtICNE',
  'MtInt778',
  'MtIntExer',
  'MtPrimePayeeCouv',
  'MtProduits',
  'MtSortie',
  'NatCouv',
  'NomBenefEmprGaranti',
  'NumContrat',
  'NumContratCouv',
  'ObjEmpr',
  'PartGarantie',
  'ProfilAmort',
  'ProfilAmortDtVote',
  'ProvGarantiEmpr',
  'RReelFon',
  'Renegocie',
  'RtAnticipe',
  'Structure',
  'StructureDtVote',
  'StuctureApresCouv',
  'StuctureAvantCouv',
  'Tot1Annuite',
  'TotGarEchoirExer',
  'TxActua',
  'TxActuaInit',
  'TxActuaReneg',
  'TxApresCouv',
  'TxMargeInit',
  'TxMaxi',
  'TxMini',
  'TxPaye',
  'TxRecu',
  'Txinit',
  'TypCouv',
  'TypeSortie']

In [37]:
df_emprunt.json_annexe

0         [{"CodTypEmpr": "10", "RReelFon": "7125.00"}]
1     [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
2     [{"CodTypEmpr": "01", "CodProfilAmort": "C", "...
3     [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
4     [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
                            ...                        
95    [{"CodTypEmpr": "01", "CodProfilAmort": "C", "...
96    [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
97       [{"CodTypEmpr": "10", "RReelFon": "98356.50"}]
98    [{"CodTypEmpr": "01", "CodProfilAmort": "X", "...
99    [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...
Name: json_annexe, Length: 100, dtype: object

In [36]:
dfs = []
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))
print(json_df)
json_df.assign(**row)
#print('dfs')
#print(dfs)

   CodTypEmpr CodProfilAmort CodProfilAmortDtVote CodArticle AnEncaisse  \
0          01              P                    P       1641       2006   
1          01              P                    P       1641       2008   
2          01              P                    P       1641       2008   
3          01              P                    P       1641       2010   
4          01              P                    P       1641       2010   
5          01              P                    P       1641       2009   
6          09              P                    P       1641       1993   
7          09              P                    P       1641       1994   
8          09              P                    P       1641       1993   
9          09              P                    P       1641       1998   
10         09              P                    P       1641       2007   
11         09              P                    P       1641       2010   
12         09            

,CodTypEmpr,CodProfilAmort,CodProfilAmortDtVote,CodArticle,AnEncaisse,ObjEmpr,MtEmprOrig,DureeRest,CodTypPreteur,LibOrgaPreteur,...,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,01,P,P,1641,2006,CONTRAT N 99141034347 TRAVAUX D'INVESTISSEMENT...,200000.00,5.99,01,CREDIT AGRICOLE NORD DE FRANCE,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
1,01,P,P,1641,2008,CONTRAT N 99143369010 TRAVAUX D'INVESTISSEMENT...,300000.00,7.28,01,CREDIT AGRICOLE NORD DE FRANCE,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
2,01,P,P,1641,2008,CONTRAT N99144087638 TRAVAUX INVESTISSEMENT 2008,300000.00,2.99,01,CREDIT AGRICOLE NORD DE FRANCE,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
3,01,P,P,1641,2010,CONTRAT N°MON270203 REFINANCEMENT,534395.96,14.99,01,CAISSE FRANCAISE DE FINANCEMENT,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
4,01,P,P,1641,2010,CONTRAT N 18312001,160000.00,14.99,01,SOCIETE GENERALE,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
5,01,P,P,1641,2009,CONTRAT N MON264990EUR,300000.00,3.99,01,CAISSE FRANCAISE DE FINANCEMENT,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
6,09,P,P,1641,1993,CONTRAT N° 0277588,593392.55,5.40,None,CAISSE DES DEPOTS ET CONSIGNATIONS,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
7,09,P,P,1641,1994,CONTRAT N° 0358539,585480.45,6.40,None,CAISSE DES DEPOTS ET CONSIGNATIONS,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
8,09,P,P,1641,1993,CONTRAT N° 0414184,54957.87,5.40,None,CAISSE DES DEPOTS ET CONSIGNATIONS,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES
9,09,P,P,1641,1998,CONTRAT N° 0478102,279103.66,9.98,None,CAISSE DES DEPOTS ET CONSIGNATIONS,...,21590592800010,Principal,M14-M14_COM_SUP3500,2020,Compte administratif,3,Principal,21590592800010,MAIRIE DE THUMERIES,MAIRIE DE THUMERIES


- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

In [33]:
def generate_csv(df, name):
    csv_field = df.to_csv(name)
    return csv_field
generate_csv(df_emprunt, "emprunt.csv")

In [34]:
# 3.4 - avec f string
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

def get_data(connexion, annexe, year, numero_docbudge):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '{numero_docbudge}' 
        LIMIT 5 """
    dat = pd.read_sql_query(query, conn)
    return dat
get_data(connexion=conn, annexe='DATA_EMPRUNT', year=2019, numero_docbudge= '09')

/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,213983,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",37243,21240158200011,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_FORMATION, ...",37243,DUSSAC,24158,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6336"", ""ContNat"": ""012"", ""ArtSpe""...",21240158200011,21240158200011,DUSSAC,Commune,24
1,214122,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",37269,24320045800012,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",37269,Principal,32006096,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""001"", ""Fonction"": ""020"", ""ArtSpe""...",24320045800012,24320045800012,CCGA,CCGA,32
2,214314,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",37302,21550468900013,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",37302,Principal,55468,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""023"", ""ArtSpe"": ""false"", ""CodRD"":...",21550468900013,21550468900013,COMMUNE DE SAMOGNEUX,COMMUNE DE SAMOGNEUX,55
3,214519,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""14873.48""}]",37342,26060091100010,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",37342,CCAS,None,M14-M14_CCAS_INF3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ArtSpe"": ""false"", ""CodRD"":...",26060091100010,26060091100010,COMMUNE AURIBEAU/S.,COMMUNE,06
4,214844,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""652.00""}]",37406,29830029400014,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",37406,ASA LA FOUX,None,M14-M14_COM_INF500,...,None,1,P,None,"[{""Nature"": ""6161"", ""ContNat"": ""011"", ""ArtSpe""...",29830029400014,29830029400014,ASA LA FOUX,ASSOSYNDAUTORIS,83


In [ ]:
annexe_dic = dict()
for item in data:
    annexe_dic[item] =  get_data(connexion=conn, annexe=item, year=2020, numero_docbudge= '09')

In [36]:
annexe_dic["DATA_EMPRUNT"]

,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,214850,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",37407,21540163900012,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",37407,COMMUNE DE DOMJEVIN,54 163,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""023"", ""ArtSpe"": ""false"", ""CodRD"":...",21540163900012,21540163900012,COMMUNE DE DOMJEVIN,COMMUNE DE DOMJEVIN,54
1,215423,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""TotGarEchoirExer"": ""167...",37506,22700001500015,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONCOURS, DATA...",37506,Budget principal,70,M52-M52,...,None,3,P,22700001500015,"[{""Nature"": ""13931"", ""Fonction"": ""01"", ""ContNa...",22700001500015,22700001500015,Département de la Haute-Saône,Départements,None
2,214858,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""20000.00""}]",37408,21240152500101,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",37408,GENDARMERIE,24152,M14-M14_COM_500_3500,...,None,1,A,21240152500010,"[{""Nature"": ""60632"", ""ContNat"": ""011"", ""ArtSpe...",21240152500101,21240152500101,DOMME,Commune,24
3,214882,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""33073.90""}]",37413,26380316500012,"[DATA_EMPRUNT, DATA_FORMATION, DATA_CONSOLIDAT...",37413,CCAS DE SAINT ETIENNE DE SAINT GEOIRS,38384,M14-M14_CCAS_INF3500,...,None,1,P,None,"[{""Nature"": ""60623"", ""ContNat"": ""011"", ""ArtSpe...",26380316500012,26380316500012,COMMUNE DE ST ETIENNE DE ST GEOIRS,Commune,38
4,214904,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",37417,21120228800018,"[DATA_EMPRUNT, DATA_PERSONNEL, DATA_SOMMAIRE]",37417,SAINT IZAIRE,12110228,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""020"", ""LibCpte"": ""D\u00e9penses i...",21120228800018,21120228800018,SAINT IZAIRE,Commune,12


In [37]:
for item in annexe_dic:
    print(annexe_dic[item])

Empty DataFrame
Columns: [id, type_annexe, json_annexe, fk_id_document_budgetaire, siret_etablissement, list_annexes, id, libelle, code_insee, nomenclature, exercice, nature_dec, num_dec, nature_vote, type_budget, id_etabl_princ, json_budget, fk_siret_collectivite, siret_coll, libelle_collectivite, nature_collectivite, departement]
Index: []

[0 rows x 22 columns]
       id   type_annexe                                        json_annexe  \
0  214850  DATA_EMPRUNT  [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...   
1  215423  DATA_EMPRUNT  [{"CodTypEmpr": "10", "TotGarEchoirExer": "167...   
2  214858  DATA_EMPRUNT     [{"CodTypEmpr": "10", "RReelFon": "20000.00"}]   
3  214882  DATA_EMPRUNT     [{"CodTypEmpr": "10", "RReelFon": "33073.90"}]   
4  214904  DATA_EMPRUNT  [{"CodTypEmpr": "01", "CodProfilAmort": "P", "...   

   fk_id_document_budgetaire siret_etablissement  \
0                      37407      21540163900012   
1                      37506      22700001500015   
2       

In [41]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres", 
                        port = "5432")
query = """
 SELECT * from annexe 
 JOIN documentbudgetaire 
     ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
 JOIN collectivite 
     ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
 WHERE annexe.type_annexe = 'DATA_EMPRUNT' 
 AND documentbudgetaire.exercice = '2020'
 AND documentbudgetaire.nature_dec = '09' 
 LIMIT 100
 """
dat = pd.read_sql_query(query, conn)
conn.close()



/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [42]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [115]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(dat.columns) if item not in interesting_columns]
df = dat.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

In [7]:
df_min

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",21630291900016,Budget Communal M14,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21630291900016,Commune de PUY-GUILLAUME,Commune
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""09"", ""CodProfilAmort"": ""P"", ""...",21750001600019,BUDGET PRINCIPAL,M57-M57,2020,Compte administratif,2,Principal,21750001600019,Ville de Paris,Commune
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""51265.65""}]",21630463400043,BUDGET EAU,M4-M49_D,2020,Compte administratif,1,Annexe,21630463400043,Commune de Viscomtat,Commune
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""50493.40""}]",26390475700014,Budget CCAS 42000,M14-M14_CCAS_INF3500,2020,Compte administratif,1,Principal,26390475700014,Commune de SAINT AMOUR,Commune
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21330331600011,74000 BUDGET COMMUNAL M14,M14-M14_COM_INF500,2020,Compte administratif,1,Principal,21330331600011,MAIRIE DE PONDAURAT,Commune


In [119]:
dfs = []
#EMPRUNT_FIELD = interesting_columns
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))
    print(json_df)
    #json_df.assign(**row)

  CodTypEmpr CodProfilAmort CodProfilAmortDtVote CodArticle AnEncaisse  \
0         01              P                    P      16441       2006   
1         01              P                    P      16441       2007   

  MtEmprOrig DureeRest CodTypPreteur    LibOrgaPreteur CodPeriodRemb  ...  \
0  740000.00      0.05            01  SOCIETE GENERALE             I  ...   
1  370000.00      1.98            01  SOCIETE GENERALE             I  ...   

  RReelFon StuctureAvantCouv Tot1Annuite TotGarEchoirExer TxActuaReneg  \
0     None              None        None             None         None   
1     None              None        None             None         None   

  TxMargeInit TxPaye TxRecu TypCouv TypeSortie  
0        None   None   None    None       None  
1        None   None   None    None       None  

[2 rows x 97 columns]
  CodTypEmpr CodProfilAmort CodProfilAmortDtVote           ProfilAmort  \
0         01              X                    X  Echéance progressive   
1   

In [15]:
df_to_csv = pd.concat(dfs)
df_to_csv.to_csv('./csv/2020-CA-Limit100.csv', index=False)  

In [9]:
EMPRUNT_FIELD = ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv',
  'IndSousJacentDtVote',
  'IndexTxVariDtVote',
  'IndexTxVariInit',
  'IndexTxVariReneg',
  'IndiceCouv',
  'IndiceEmpr',
  'IndiceEmprDtVote',
  'LibCpte',
  'LibOrgCoContr',
  'LibOrgaPreteur',
  'MPrimeRecueCouv',
  'MtCRDCouvert',
  'MtCRDRefin',
  'MtCapitalExer',
  'MtCapitalReamenage',
  'MtCapitalRestDu_01_01',
  'MtCapitalRestDu_31_12',
  'MtCharges',
  'MtCommCouv',
  'MtCouv',
  'MtCouvert',
  'MtEmprOrig',
  'MtEmprReneg',
  'MtICNE',
  'MtInt778',
  'MtIntExer',
  'MtPrimePayeeCouv',
  'MtProduits',
  'MtSortie',
  'NatCouv',
  'NomBenefEmprGaranti',
  'NumContrat',
  'NumContratCouv',
  'ObjEmpr',
  'PartGarantie',
  'ProfilAmort',
  'ProfilAmortDtVote',
  'ProvGarantiEmpr',
  'RReelFon',
  'Renegocie',
  'RtAnticipe',
  'Structure',
  'StructureDtVote',
  'StuctureApresCouv',
  'StuctureAvantCouv',
  'Tot1Annuite',
  'TotGarEchoirExer',
  'TxActua',
  'TxActuaInit',
  'TxActuaReneg',
  'TxApresCouv',
  'TxMargeInit',
  'TxMaxi',
  'TxMini',
  'TxPaye',
  'TxRecu',
  'Txinit',
  'TypCouv',
  'TypeSortie']